In [6]:
# Importamos las librerías que necesitamos
import tqdm as tddm
from bs4 import BeautifulSoup
import time
from time import sleep
import pandas as pd
import numpy as np
import random

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [7]:
csv_municipios = pd.read_csv("./datos/df_codigos.csv", index_col = 0)
lista_codigos = csv_municipios["index"].tolist()

In [ ]:
lista_codigos

In [2]:
def dataframe_mes(sopa, municipio, mes):
    """
    Procesa los datos climáticos de una tabla HTML (extraída con BeautifulSoup) para un municipio y mes específicos,
    separando las columnas de temperaturas, punto de rocío, humedad, velocidad del viento y presión en columnas de 
    valores máximos, promedios y mínimos.

    Args:
        sopa (BeautifulSoup): Objeto de BeautifulSoup que contiene las tablas HTML con los datos climáticos.
        municipio (str): Nombre del municipio al que corresponden los datos climáticos.
        mes (str): Nombre o número del mes correspondiente a los datos climáticos.

    Returns:
        pandas.DataFrame: DataFrame con los datos procesados, incluyendo:
                          - Municipio
                          - Mes
                          - T Max, T Avg, T Min (Temperatura)
                          - DP Max, DP Avg, DP Min (Punto de rocío)
                          - H Max, H Avg, H Min (Humedad)
                          - W Max, W Avg, W Min (Velocidad del viento)
                          - P Max, P Avg, P Min (Presión)
    """
    lista_df = []

    # Extraer filas de cada tabla y convertirlas en DataFrame
    for tabla in sopa:
        df = pd.DataFrame([tabla.findChildren('tr')[i].text for i in range(len(tabla.findChildren('tr')))])
        lista_df.append(df)

    # Concatenar las tablas
    df_final = pd.concat(lista_df, axis=1)

    # Eliminar la primera fila (asumiendo que contiene encabezados o datos irrelevantes)
    df_final = df_final.iloc[1:]

    # Función auxiliar para separar columnas en múltiples valores
    def separar_columnas(df, columna, nuevas_columnas):
        df[nuevas_columnas] = df[columna].str.split(expand=True)
        df.drop(columns=[columna], inplace=True)

    # Añadir columnas de Municipio y Mes
    df_final.insert(loc=0, column="Municipio", value=municipio)
    df_final.insert(loc=1, column="Mes", value=mes)

    # Separar columnas en máximos, promedios y mínimos
    separar_columnas(df_final, 'Temperature', ['T Max', 'T Avg', 'T Min'])
    separar_columnas(df_final, 'Dew Point', ['DP Max', 'DP Avg', 'DP Min'])
    separar_columnas(df_final, 'Humidity', ['H Max', 'H Avg', 'H Min'])
    separar_columnas(df_final, 'Wind Speed', ['W Max', 'W Avg', 'W Min'])
    separar_columnas(df_final, 'Pressure', ['P Max', 'P Avg', 'P Min'])

    return df_final

In [ ]:
driver = webdriver.Chrome()
url_wunder = "https://www.wunderground.com/history"
driver.get(url_wunder)
time.sleep(2)

try:
    iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located(('xpath', '//*[@id="sp_message_iframe_1165301"]')))
    driver.switch_to.frame(iframe)
    driver.find_element("css selector", "#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()
    driver.switch_to.default_content()
    time.sleep(5)
except:
    print('Botón cookies no encontrado')

#lista_codigos

lista_tablas_municipios = []

for municipio in lista_codigos:
    try:
        driver.find_element("css selector", "#historySearch").send_keys(municipio, Keys.TAB)
        time.sleep(3)
        driver.find_element("css selector", "#dateSubmit").click()
        time.sleep(3)
        driver.find_element("css selector", "#dateSubmit").click()
        time.sleep(3)
        driver.find_element("css selector", "#inner-content > div.region-content-main > div.row > div:nth-child(1) > div:nth-child(1) > div > lib-link-selector > div > div > div > a:nth-child(3)").click()
        time.sleep(3)
    except:
        print("Ha ocurrido un error al intentar ingresar a la URL")
    
    url_iteracion = driver.current_url
    print(f"Iterando sobre {url_iteracion}")
    lista_urls = [url_iteracion[:-2] + str(mes) for mes in range(1, 11)]
    lista_tablas = []
    
    for url in lista_urls:
        try:
            driver.get(url)
            mes = url[-2:]
            time.sleep(2)
            html_table_page = driver.page_source
            sopa = BeautifulSoup(html_table_page, 'html.parser')
            tabla = sopa.findAll("table", {'aria-labelledby': 'Days data'})
            if not tabla:
                print(f'Tabla no encontrada para: {url}')
                continue
            
            tabla1 = dataframe_mes(tabla, municipio, mes)
            if tabla1 is not None:
                lista_tablas.append(tabla1)
            else:
                print(f'Ha ocurrido un error para el municipio {municipio} en {mes}')
        except Exception as e:
            print(f"Error al procesar la URL {e}")
    
    if lista_tablas:
        df_unido = pd.concat(lista_tablas, ignore_index=True)
        lista_tablas_municipios.append(df_unido)
        driver.get(url_wunder)
    else:
        print(f'No hay datos para el municipio {municipio}')
        driver.get(url_wunder)

df_completo = pd.concat(lista_tablas_municipios, ignore_index=True)
